In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [212]:
import pyspark

In [ ]:
import pandas as pd
pd.read_csv('info.csv')

,Name,Age
0,T,14
1,H,52
2,D,7
3,A,8
4,B,16
5,N,45
6,M,3
7,L,26
8,K,64
9,Q,92


In [ ]:
from pyspark.sql import SparkSession 

In [ ]:
spark=SparkSession.builder.appName('Partise').getOrCreate()

In [ ]:
spark

In [ ]:
df_spark=spark.read.option("header", True).csv('info.csv')
df_spark.show()

+----+---+
|Name|Age|
+----+---+
|   T| 14|
|   H| 52|
|   D|  7|
|   A|  8|
|   B| 16|
|   N| 45|
|   M|  3|
|   L| 26|
|   K| 64|
|   Q| 92|
|   S| 19|
|   P| 30|
|   Y| 29|
|   G| 34|
+----+---+



In [ ]:
df_spark.select('Age').show()

+---+
|Age|
+---+
| 14|
| 52|
|  7|
|  8|
| 16|
| 45|
|  3|
| 26|
| 64|
| 92|
| 19|
| 30|
| 29|
| 34|
+---+



In [ ]:
df_spark.createOrReplaceTempView("df_spark")

In [63]:
!wget https://raw.githubusercontent.com/demanejar/retails/master/resources/retails.csv

--2023-04-05 16:12:32--  https://raw.githubusercontent.com/demanejar/retails/master/resources/retails.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43954909 (42M) [text/plain]
Saving to: ‘retails.csv’

retails.csv         100%[===================>]  41.92M   264MB/s    in 0.2s    

2023-04-05 16:12:33 (264 MB/s) - ‘retails.csv’ saved [43954909/43954909]



In [118]:
df=SparkSession.builder.appName('Part1').getOrCreate()
df=df.read.option('header', True).csv('retails.csv')
df.show()

+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

Part 1

Câu hỏi: Có tổng bao nhiêu giao dịch, sản phẩm và khách hàng khác nhau?

In [165]:
flag=df.select("CustomerID").distinct().count()-1
print("Khách hàng : ", flag)

Khách hàng :  4372


In [166]:
flag=df.select("StockCode").distinct().count()
print("Sản phẩm : ", flag)

Sản phẩm :  4070


In [167]:
flag=df.select("InvoiceNo").distinct().count()
print("Giao dịch : ", flag)

Giao dịch :  25900


In [226]:
from pyspark.sql.functions import countDistinct

# Đếm số lượng giao dịch khác nhau
num_transactions = df.select(countDistinct('InvoiceNo')).collect()[0][0]

# Đếm số lượng sản phẩm khác nhau
num_products = df.select(countDistinct('StockCode')).collect()[0][0]

# Đếm số lượng khách hàng khác nhau
num_customers = df.select(countDistinct('CustomerID')).collect()[0][0]

print(f"Tổng số giao dịch: {num_transactions}")
print(f"Tổng số sản phẩm: {num_products}")
print(f"Tổng số khách hàng: {num_customers}")

Tổng số giao dịch: 25900
Tổng số sản phẩm: 4070
Tổng số khách hàng: 4372


Part 2

Câu hỏi: Tỉ lệ khách hàng không có thông *tin*

In [224]:
flag =df.select("CustomerID").filter(df["CustomerID"].isNull()).count()/float(df.select("CustomerID").distinct().count())
print("Tỉ lệ khách hàng không có thông tin : ", flag)

Tỉ lệ khách hàng không có thông tin :  30.889549508346672


Part 3

Câu hỏi: Đâu là nước có số lượng đơn hàng (Quantity) nhiều thứ 3?

In [159]:
df=df.withColumn('Quantity', df['Quantity'].cast('int'))
df.groupBy('Country').sum('Quantity').sort('sum(Quantity)',ascending=False).show()

+---------------+-------------+
|        Country|sum(Quantity)|
+---------------+-------------+
| United Kingdom|      4263829|
|    Netherlands|       200128|
|           EIRE|       142637|
|        Germany|       117448|
|         France|       110480|
|      Australia|        83653|
|         Sweden|        35637|
|    Switzerland|        30325|
|          Spain|        26824|
|          Japan|        25218|
|        Belgium|        23152|
|         Norway|        19247|
|       Portugal|        16180|
|        Finland|        10666|
|Channel Islands|         9479|
|        Denmark|         8188|
|          Italy|         7999|
|         Cyprus|         6317|
|      Singapore|         5234|
|        Austria|         4827|
+---------------+-------------+
only showing top 20 rows



In [163]:
flag = df.groupBy('Country').sum('Quantity').sort('sum(Quantity)',ascending=False).select('Country').take(3)
print("Nước có số lượng đơn hàng (Quantity) nhiều thứ 3 : ",flag[2])

Nước có số lượng đơn hàng (Quantity) nhiều thứ 3 :  Row(Country='EIRE')


Part 4

Câu hỏi: Từ nào xuất hiện ít và nhiều nhất trong phần Description?

In [215]:
import collections
from pyspark import SparkContext
from pyspark.sql.functions import desc

flag=df.groupBy('Description').count().orderBy('count')
least_common_word = flag.first()['Description']
print("Từ xuất hiện ít nhất là:", least_common_word)
flag=df.groupBy('Description').count().orderBy(desc('count'))
most_common_word = flag.first()['Description']
print("Từ xuất hiện nhiều nhất trong phần Description là:", most_common_word)

Từ xuất hiện ít nhất là: PINK BOUDOIR T-LIGHT HOLDER
Từ xuất hiện nhiều nhất trong phần Description là: WHITE HANGING HEART T-LIGHT HOLDER


Part 5

Câu hỏi: Sản phẩm nào bán được số lượng (Quantity) lớn nhất ở United Kingdom?

In [220]:
UK = df.filter(df.Country == 'United Kingdom')
product = UK.groupBy('StockCode').sum('Quantity')
sort_sum = product.sort('sum(Quantity)', ascending=False)
most_sold_product = sort_sum.first()['StockCode']
name = df.filter(df.StockCode == most_sold_product).select('Description').distinct().collect()[0]['Description']
print('Sản phẩm bán được số lượng lớn nhất ở United Kingdom là : ', name)

Sản phẩm bán được số lượng lớn nhất ở United Kingdom là :  POPCORN HOLDER
